<a href="https://colab.research.google.com/github/rizkypratamayudha/dataMining/blob/main/ANN_Latihan_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# Load dataset Titanic
titanic_data = pd.read_csv('Titanic-Dataset.csv')

# Preprocessing data
titanic_data = titanic_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
titanic_data.dropna(inplace=True)

# Encode categorical variables
titanic_data = pd.get_dummies(titanic_data, columns=['Sex', 'Embarked'])

# Separate features and labels
x = titanic_data.drop('Survived', axis=1).values
y = titanic_data['Survived'].values

# Normalize the features
scaler = StandardScaler()
x = scaler.fit_transform(x)

# One Hot encoding untuk label
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Bagi data menjadi training dan testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Fungsi aktivasi sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Turunan fungsi sigmoid
def sigmoid_derivative(x):
    return x * (1 - x)

# Parameter jaringan neural
input_layer_neurons = x_train.shape[1]
hidden_layer_neurons = 10
output_layer_neurons = y_train.shape[1]

# Inisialisasi bobot dan bias
np.random.seed(42)
weights_input_hidden = np.random.uniform(size=(input_layer_neurons, hidden_layer_neurons))
weights_hidden_output = np.random.uniform(size=(hidden_layer_neurons, output_layer_neurons))
bias_hidden = np.random.uniform(size=(1, hidden_layer_neurons))
bias_output = np.random.uniform(size=(1, output_layer_neurons))

# Hiperparameter
learning_rate = 0.01
epochs = 5000

# Proses pelatihan
for epoch in range(epochs):
    # Forward propagation
    hidden_layer_input = np.dot(x_train, weights_input_hidden) + bias_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    predicted_output = sigmoid(output_layer_input)

    # Backpropagation
    error = y_train - predicted_output
    d_predicted_output = error * sigmoid_derivative(predicted_output)
    error_hidden_layer = d_predicted_output.dot(weights_hidden_output.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

    # Update bobot dan bias
    weights_hidden_output += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
    bias_output += np.sum(d_predicted_output, axis=0, keepdims=True) * learning_rate
    weights_input_hidden += x_train.T.dot(d_hidden_layer) * learning_rate
    bias_hidden += np.sum(d_hidden_layer, axis=0, keepdims=True) * learning_rate

    # Cetak error setiap 1000 epochs
    if (epoch + 1) % 1000 == 0:
        loss = np.mean(np.square(error))
        print(f"Epoch {epoch + 1}, Loss: {loss}")

# Prediksi pada data uji
hidden_layer_input = np.dot(x_test, weights_input_hidden) + bias_hidden
hidden_layer_output = sigmoid(hidden_layer_input)
output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
predicted_output = sigmoid(output_layer_input)

# Evaluasi model
y_test_labels = np.argmax(y_test, axis=1)
predicted_labels = np.argmax(predicted_output, axis=1)
accuracy = accuracy_score(y_test_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100}%")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1000, Loss: 0.10688749534403369
Epoch 2000, Loss: 0.09704520280637667
Epoch 3000, Loss: 0.09317259377123718
Epoch 4000, Loss: 0.09003926785060952
Epoch 5000, Loss: 0.08702326296896304
Accuracy: 72.02797202797203%
